# German Word Capitalization Prediction

### *How to create an NLP Machine Learning model for predicting noun capilaization in German?*

In [401]:
import os
import re
import gensim
import string
import numpy as np
from datetime import datetime

#### 1 Prepare Test Data

Test data ist located in files in the following directory numbers: **dir_list [ 95 : 118 ]**.

In [101]:
def clean_test_data(directory_nr, output_list):
    dir_list = os.listdir(os.getcwd() + "\\extracted\\")
    
    for filename in os.listdir(os.getcwd() + "\\extracted\\" + dir_list[directory_nr]):
        with open(os.path.join(os.getcwd() + "\\extracted\\" + dir_list[directory_nr] + "\\" + filename), 'rt', encoding="utf-8") as f:
            output_list.append(f.read())

    output_list[:] = [x.encode('utf-8').decode('utf-8') for x in output_list]

    # cleaning
    
    # removing double space, chars and line breaks
    output_list[:] = [((x.replace("\xa0", " ")).replace("\n", " ")).replace("  ", " ") for x in output_list] 

    # remove <doc> tag, link and article id in beginning
    output_list[:] = [re.sub(r'<.+?> ', '', x) for x in output_list]
    
    #return output_list[0]
  
    # remove digits, and the following words f.e. 1. August 2020 - 1. and 2020 are removed
    # output_list = [re.sub("\S*\d\S*", '', x).strip() for x in output_list]
    
    #return output_list
    
    
    

In [102]:
test_95 = []
clean_test_data(95, test_95)

In [513]:
test_95[0][0:300]

'Pazifische Taifunsaison 2015 Die Pazifische Taifunsaison 2015 ist ein andauerndes Wetterereignis, unter dem die sich während des gesamten Kalenderjahres bildenden tropischen Wirbelstürme zusammengefasst sind. Die meisten tropischen Wirbelstürme bilden sich im Pazifischen Ozean westlich der Datumsgre'

In [546]:
test_data = test_95[0].split()[:100]

In [547]:
test_data = " ".join(test_data)
test_data

'Pazifische Taifunsaison 2015 Die Pazifische Taifunsaison 2015 ist ein andauerndes Wetterereignis, unter dem die sich während des gesamten Kalenderjahres bildenden tropischen Wirbelstürme zusammengefasst sind. Die meisten tropischen Wirbelstürme bilden sich im Pazifischen Ozean westlich der Datumsgrenze und nördlich des Äquators jedoch zwischen Mai und November. Solche tropischen Wirbelstürme nennt man Taifune. Die sich östlich des 180. Längengrades bildenden Stürme sind Gegenstand der Pazifischen Hurrikansaison 2015. Innerhalb des nordwestlichen Pazifiks gibt es zwei meteorologische Organisationen, die den Stürmen Namen vergeben. Dies führt dazu, dass derselbe Sturm häufig zwei verschiedene Namen erhält. Die offizielle Bezeichnung durch die Japan Meteorological Agency (JMA) erhält'

#### 2 Test Data - Method Implementation - Built in Prediction

In [317]:
model = gensim.models.Word2Vec.load("w2v_model_train_48.bin")
lst = model.predict_output_word(["meinem", "haus", "ein" ], topn=model.wv.vectors.shape[0]) 
words = list(model.wv.vocab)

In [312]:
print("Foo",end=' ')
print('Bar')

Foo Bar


In [397]:
input_x = "ich Habe anna 5. meinem haus ein Garten mit vielen Blumen Wie rosen."
actual_x = "Ich habe Anna 5. meinem Haus ein Garten mit vielen Blumen wie Rosen."

In [501]:
x = datetime.now()



##hourMinute = datetime.now.tostring("HH:mm");
#time = datetime.strptime(x, '%H:%M:%S')# - datetime.strptime(s1, format)
x

datetime.datetime(2020, 3, 24, 20, 33, 43, 615745)

In [502]:
y = datetime.now()

c = y-x
f = str(c)
print(f)
f = f.split(".")[0]

#for c in f:
if c[0] == "0":
    print(f[2:7])
    

0:00:03.254773


TypeError: 'datetime.timedelta' object is not subscriptable

In [566]:
word_predictor(test_data, "w2v_model_train_48.bin", test_data, progress="show")

############## Prediction ############## 

Input Text:      Pazifische Taifunsaison 2015 Die Pazifische Taifunsaison 2015 ist ein andauerndes Wetterereignis, un

 predicting...   Pazifische Taifunsaison 2015 die Pazifische (predicted) Taifunsaison 2015 ist ein andauerndes Wetterereignis , unter dem die sich während des gesamten (predicted) Kalenderjahres bildenden (predicted) tropischen (predicted) Wirbelstürme zusammengefasst (predicted) sind . Die meisten (predicted) tropischen (predicted) Wirbelstürme bilden (predicted) sich im Pazifischen (predicted) Ozean westlich (predicted) der Datumsgrenze und nördlich (predicted) des Äquators jedoch (predicted) zwischen Mai (predicted) und November . Solche tropischen (predicted) Wirbelstürme nennt (predicted) man Taifune . Die sich östlich (predicted) des 180 . Längengrades bildenden (predicted) Stürme sind Gegenstand der Pazifischen (predicted) Hurrikansaison 2015 . Innerhalb des nordwestlichen (predicted) Pazifiks gibt (predicted) es zwei (

In [565]:
## beta v5 - working

def word_predictor(str_input, model_filename, evaluation = None, progress = ""):
    
    """  
    str_input.........takes any German text as input, must be a single string
    model_filename....takes a pretrained Word2Vec Embedding, will be loaded with Gensim
    evaluation........optional, takes same text as in str_input but with correct capitalization
    progress..........optional, use progress="show" for displaying prediction progress
    """
    
    global str_output, predict_with, start_time, end_time
    split_orig, split_low, split_cap, found, unknown, predict_output, predict_with = [], [], [], [], [], [], []
    
    start_time = datetime.now()
    
    print(" Prediction ".center(40, '#') ,
            "\n""\n" 
            "Input Text:     " , str_input[0:100])
    print("\n" , "predicting...", end= "   ")
    
    model = gensim.models.Word2Vec.load(model_filename)
    words = list(model.wv.vocab)
    
    split_orig = re.findall(r"\w+|[^\w\s]", str_input, re.UNICODE)
    split_in_words = [x for x in split_orig if x in words]
    split_low = [x.lower() for x in split_orig]
    split_cap = [x.capitalize() for x in split_orig]   

    l = 0
    while l < (len(split_orig)):
        for i in split_orig:
            if i in string.punctuation:
                predict_output.append(i)
                if progress=="show":
                    print(i, end=" ")
                l += 1
            elif l == 0:
                predict_output.append(i.capitalize())
                if progress=="show":
                    print(i.capitalize(), end= " ")
                l += 1
            elif predict_output[-1] == ".":
                predict_output.append(i.capitalize())
                if progress=="show":
                    print(i.capitalize(), end= " ")
                l += 1
            else:
                if split_low[l] in words:
                    if split_cap[l] in words:
                        if len(found) > 0:
                            if len(split_in_words) >= (l+4):
                                predict_with = [] + found
                                predict_with.append(split_in_words[l+1])
                                predict_with.append(split_in_words[l+2])    
                                predict_with.append(split_in_words[l+3])

                                predict_next = model.predict_output_word(predict_with[0:len(predict_with)], topn=model.wv.vectors.shape[0]) 

                                for k in predict_next:
                                    if split_low[l] == k[0]:
                                        word_low = k[0]
                                        val_low = k[1]
                                for k in predict_next:
                                    if k[0] == split_cap[l]:
                                        word_cap = k[0]
                                        val_cap = k[1]                                                

                                if val_low >= val_cap:
                                    predict_output.append(i.lower())
                                    found.append(i.lower())
                                    if progress=="show":
                                        print(i.lower(), "(predicted)", end= " ")
                                    l += 1 
                                elif val_low <= val_cap:
                                    predict_output.append(i.capitalize())
                                    found.append(i.capitalize())
                                    if progress=="show":
                                        print(i.capitalize(), "(predicted)", end= " ")
                                    l += 1  
                                else:                                                                         
                                    predict_output.append(i.upper())
                                    unknown.append(i.upper())
                                    if progress=="show":
                                        print(i.upper(), end= " ")
                                    l += 1                                    

                            elif len(split_in_words) >= (l+3):
                                predict_with = [] + found
                                predict_with.append(split_in_words[l+1])
                                predict_with.append(split_in_words[l+2])    

                                predict_next = model.predict_output_word(predict_with[0:len(predict_with)], topn=model.wv.vectors.shape[0]) 

                                for k in predict_next:
                                    if split_low[l] == k[0]:
                                        word_low = k[0]
                                        val_low = k[1]
                                for k in predict_next:
                                    if k[0] == split_cap[l]:
                                        word_cap = k[0]
                                        val_cap = k[1]                                                

                                if val_low >= val_cap:
                                    predict_output.append(i.lower())
                                    found.append(i.lower())
                                    if progress=="show":
                                        print(i.lower(), "(predicted)", end= " ")
                                    l += 1 
                                elif val_low <= val_cap:
                                    predict_output.append(i.capitalize())
                                    found.append(i.capitalize())
                                    if progress=="show":
                                        print(i.capitalize(), "(predicted)", end= " ")
                                    l += 1  
                                else:                                                                         
                                    predict_output.append(i.upper())
                                    unknown.append(i.upper())
                                    if progress=="show":
                                        print(i.upper(), end= " ")
                                    l += 1                                              

                            elif len(split_in_words) > (l+1): # predict with l+1
                                predict_with = [] + found
                                predict_with.append(split_in_words[l+1])
                                predict_next = model.predict_output_word(predict_with[0:len(predict_with)], topn=model.wv.vectors.shape[0]) 

                                for k in predict_next:
                                    if split_low[l] == k[0]:
                                        word_low = k[0]
                                        val_low = k[1]
                                for k in predict_next:
                                    if k[0] == split_cap[l]:
                                        word_cap = k[0]
                                        val_cap = k[1]                                                

                                if val_low >= val_cap:
                                    predict_output.append(i.lower())
                                    found.append(i.lower())
                                    if progress=="show":
                                        print(i.lower(), "(predicted)", end= " ")
                                    l += 1 
                                elif val_low <= val_cap:
                                    predict_output.append(i.capitalize())
                                    found.append(i.capitalize())
                                    if progress=="show":
                                        print(i.capitalize(), "(predicted)", end= " ")
                                    l += 1  
                                else:                                                                         
                                    predict_output.append(i.upper())
                                    unknown.append(i.upper())
                                    if progress=="show":
                                        print(i.upper(), end= " ")
                                    l += 1       

                            else:
                                if len(found) >= 10:
                                    predict_next = model.predict_output_word(found[-10:len(found)], topn=model.wv.vectors.shape[0]) 

                                    for k in predict_next:
                                        if split_low[l] == k[0]:
                                            word_low = k[0]
                                            val_low = k[1]
                                    for k in predict_next:
                                        if split_cap[l] == k[0]:
                                            word_cap = k[0]
                                            val_cap = k[1]

                                    if val_low >= val_cap:
                                        predict_output.append(i.lower())
                                        found.append(i.lower())
                                        if progress=="show":
                                            print(i.lower(), "(predicted)", end= " ")
                                        l += 1 
                                    elif val_low <= val_cap:
                                        predict_output.append(i.capitalize())
                                        found.append(i.capitalize())
                                        if progress=="show":
                                            print(i.capitalize(), "(predicted)", end= " ")
                                        l += 1  
                                    else:                                                                         
                                        predict_output.append(i.upper())
                                        unknown.append(i.upper())
                                        if progress=="show":
                                            print(i.upper(), end= " ")
                                        l += 1

                                elif len(found) >= 5:  
                                    # predict only with the last 5
                                    predict_next = model.predict_output_word(found[-5:len(found)], topn=model.wv.vectors.shape[0]) 

                                    for k in predict_next:
                                        if split_low[l] == k[0]:
                                            word_low = k[0]
                                            val_low = k[1]

                                    for k in predict_next:
                                        if split_cap[l] == k[0]:
                                            word_cap = k[0]
                                            val_cap = k[1]

                                    if val_low >= val_cap:
                                        predict_output.append(i.lower())
                                        found.append(i.lower())
                                        if progress=="show":
                                            print(i.lower(), "(predicted)", end= " ")
                                        l += 1 
                                    elif val_low <= val_cap:
                                        predict_output.append(i.capitalize())
                                        found.append(i.capitalize())
                                        if progress=="show":
                                            print(i.capitalize(), "(predicted)", end= " ")
                                        l += 1  
                                    else:                                                                         
                                        predict_output.append(i.upper())
                                        unknown.append(i.upper())
                                        if progress=="show":
                                            print(i.upper(), end= " ")
                                        l += 1    

                                else:
                                    predict_next = model.predict_output_word(found[0:len(found)], topn=model.wv.vectors.shape[0]) 

                                    for k in predict_next:
                                        if split_low[l] == k[0]:
                                            word_low = k[0]
                                            val_low = k[1]

                                    for k in predict_next:
                                        if split_cap[l] == k[0]:
                                            word_cap = k[0]
                                            val_cap = k[1]

                                    if val_low >= val_cap:
                                        predict_output.append(i.lower())
                                        found.append(i.lower())
                                        if progress=="show":
                                            print(i.lower(), "(predicted)", end= " ")
                                        l += 1 
                                    elif val_low <= val_cap:
                                        predict_output.append(i.capitalize())
                                        found.append(i.capitalize())
                                        if progress=="show":
                                            print(i.capitalize(), "(predicted)", end= " ")
                                        l += 1  
                                    else:                                                                         
                                        predict_output.append(i.upper())
                                        unknown.append(i.upper())
                                        if progress=="show":
                                            print(i.upper(), end= " ")
                                        l += 1    

                        else:
                            if len(split_wo_punc) >= (l+4):
                                predict_with = [] + found
                                predict_with.append(split_in_words[l+1])
                                predict_with.append(split_in_words[l+2])    
                                predict_with.append(split_in_words[l+3])

                                predict_next = model.predict_output_word(predict_with[0:len(predict_with)], topn=model.wv.vectors.shape[0]) 

                                for k in predict_next:
                                    if split_low[l] == k[0]:
                                        word_low = k[0]
                                        val_low = k[1]
                                for k in predict_next:
                                    if k[0] == split_cap[l]:
                                        word_cap = k[0]
                                        val_cap = k[1]                                                

                                if val_low >= val_cap:
                                    predict_output.append(i.lower())
                                    found.append(i.lower())
                                    if progress=="show":
                                        print(i.lower(), "(predicted)", end= " ")
                                    l += 1 
                                elif val_low <= val_cap:
                                    predict_output.append(i.capitalize())
                                    found.append(i.capitalize())
                                    if progress=="show":
                                        print(i.capitalize(), "(predicted)", end= " ")
                                    l += 1  
                                else:                                                                         
                                    predict_output.append(i.upper())
                                    unknown.append(i.upper())
                                    if progress=="show":
                                        print(i.upper(), end= " ")
                                    l += 1          

                            elif len(split_wo_punc) >= (l+3):    
                                predict_with = [] + found
                                predict_with.append(split_in_words[l+1])
                                predict_with.append(split_in_words[l+2])    

                                predict_next = model.predict_output_word(predict_with[0:len(predict_with)], topn=model.wv.vectors.shape[0]) 

                                for k in predict_next:
                                    if split_low[l] == k[0]:
                                        word_low = k[0]
                                        val_low = k[1]
                                for k in predict_next:
                                    if k[0] == split_cap[l]:
                                        word_cap = k[0]
                                        val_cap = k[1]                                                

                                if val_low >= val_cap:
                                    predict_output.append(i.lower())
                                    found.append(i.lower())
                                    if progress=="show":
                                        print(i.lower(), "(predicted)", end= " ")
                                    l += 1 
                                elif val_low <= val_cap:
                                    predict_output.append(i.capitalize())
                                    found.append(i.capitalize())
                                    if progress=="show":
                                        print(i.capitalize(), "(predicted)", end= " ")
                                    l += 1  
                                else:                                                                         
                                    predict_output.append(i.upper())
                                    unknown.append(i.upper())
                                    if progress=="show":
                                        print(i.upper(), end= " ")
                                    l += 1                                                

                            elif len(split_wo_punc) > (l+1): # predict with l+1                  
                                predict_with = [] + found
                                predict_with.append(split_in_words[l+1])  

                                predict_next = model.predict_output_word(predict_with[0:len(predict_with)], topn=model.wv.vectors.shape[0]) 

                                for k in predict_next:
                                    if split_low[l] == k[0]:
                                        word_low = k[0]
                                        val_low = k[1]

                                for k in predict_next:
                                    if k[0] == split_cap[l]:
                                        word_cap = k[0]
                                        val_cap = k[1]                                                

                                if val_low >= val_cap:
                                    predict_output.append(i.lower())
                                    found.append(i.lower())
                                    if progress=="show":
                                        print(i.lower(), "(predicted)", end= " ")
                                    l += 1 
                                elif val_low <= val_cap:
                                    predict_output.append(i.capitalize())
                                    found.append(i.capitalize())
                                    if progress=="show":
                                        print(i.capitalize(), "(predicted)", end= " ")
                                    l += 1  
                                else:                                                                         
                                    predict_output.append(i.upper())
                                    unknown.append(i.upper())
                                    if progress=="show":
                                        print(i.upper(), end= " ")
                                    l += 1                                               
                            else:                                                                      
                                predict_output.append(i.upper())
                                unknown.append(i.upper())
                                if progress=="show":
                                    print(i.upper(), end= " ")
                                l += 1

                    else:
                        predict_output.append(i.lower())
                        found.append(i.lower())
                        if progress=="show":
                            print(i.lower(), end= " ")
                        l += 1                           


                elif split_cap[l] in words:
                    predict_output.append(i.capitalize())
                    found.append(i.capitalize())
                    if progress=="show":
                        print(i.capitalize(), end=" ")
                    l += 1
                else: 
                    predict_output.append(i.upper())
                    unknown.append(i.upper())
                    if progress=="show":
                        print(i.upper(), end= " ")
                    l += 1

    end_time = datetime.now()
    str_output = " ".join(predict_output)
    str_output = re.sub(r'\s([?:.!"](?:\s|$))', r'\1', str_output)
    str_output = str_output.replace('" ', ' ')

    if evaluation != None:
        return evaluate(str_input, predict_output, evaluation)

    words_predict = [x for x in input_x.split() if x not in string.punctuation]

    print("\n""\n"
          "Predicted Text: " , str_output[0:100] ,
          "\n""\n")  

In [500]:
x = "t"

x[0:2]

't'

In [508]:
def evaluate(str_input, predict_output, actual_output):
    eval_predict, eval_act, evaluation, FP_words, FN_words = [], [], [], [], []
    actual_output = re.findall(r"\w+|[^\w\s]", actual_output, re.UNICODE)

    time_elapsed = end_time - start_time
    time_elapsed = str(time_elapsed)
    time_elapsed = time_elapsed.split(".")[0] 
    
    l = 0
    while l < len(predict_output):
        for i in predict_output:
            if i in string.punctuation:
                eval_predict.append(5)
                l += 1
            elif i == i.capitalize():
                eval_predict.append(1)
                l += 1
            elif i == i.lower():
                eval_predict.append(0)
                l += 1
            else:
                eval_predict.append(5)
                l += 1
    l = 0
    while l < len(actual_output):
        for i in actual_output:
            if i in string.punctuation:
                eval_act.append(5)
                l += 1
            elif i == i.capitalize():
                eval_act.append(1)
                l += 1
            elif i == i.lower():
                eval_act.append(0)
                l += 1
            else:
                eval_act.append(5)
                l += 1  
    l = 0            
    while l < (len(eval_predict)):
        for i in eval_predict:
            if eval_predict[l] == 1 and eval_act[l] == 1:
                evaluation.append(1)
                l += 1
            elif eval_predict[l] == 1 and eval_act[l] == 0:  
                evaluation.append(2)
                l += 1
            elif eval_predict[l] == 0 and eval_act[l] == 0:
                evaluation.append(3)
                l += 1                        
            elif eval_predict[l] == 0 and eval_act[l] == 1:
                evaluation.append(4)
                l += 1
            elif eval_predict[l] == 5 and eval_act[l] == 5:
                evaluation.append(5)
                l += 1
            else:
                evaluation.append(6)
                l += 1
                
    TP = evaluation.count(1)
    FP = evaluation.count(2)
    TN = evaluation.count(3)
    FN = evaluation.count(4)
    
    T_total = TP+TN
    F_total = FP+FN
    total = TP+TN+FP+FN
    
    acc = (round(((T_total/total)*100), 4))
    
    index_FP = [int(i) for i,x in enumerate(evaluation) if x == 2]
    index_FN = [int(i) for i,x in enumerate(evaluation) if x == 4]
    
    l = 0
    while l < len(index_FP):
        for i in index_FP:
            FP_words.append(predict_output[i])
            l += 1
    l = 0        
    while l < len(index_FN):
        for i in index_FN:
            FN_words.append(predict_output[i])
            l += 1
            
    print(  "\n""\n"
            "Predicted Text: " , str_output[0:100] ,
            "\n""\n")
    print(" Evaluation ".center(40, '#') ,
            "\n""\n" 
            "Total of " , (TP+TN+FN+FP) , " Words Predicted in " , time_elapsed ,
            "\n""\n"    
            f'{"True Positive":23} ==> {TP:5d}' ,
            "\n" 
            f'{"False Positive":23} ==> {FP:5d}' ,
            "\n"
            f'{"True Negative":23} ==> {TN:5d}' ,
            "\n"
            f'{"False Negative":23} ==> {FN:5d}' ,
            "\n""\n"  
            "Accuracy:", acc , "%" ,
            "\n""\n"
            "False Positive Words:", FP_words[0:10] ,
            "\n"
            "False Negative Words:", FN_words[0:10] )     


#### Result measurement - Accuracy

#### 4 Test Data - Method Implementation - most_similar method

In [199]:
test = [1,2,3,4,2]

#test[1][0]

word = "hello"
size = 10

print(" Evaluation ".center(40, '#') ,
      "\n\n" ,
      f'{"Total Words Predicted":20} ==> {(len(eval_predict)-eval_predict.count(5)):5d}' ,
      f'{"Prediction Time":20} ==> {time_elapsed:5d}' ,
      "\n" 
      f'{"True Positive":20} ==> {TP:5d}' ,
      f'{"False Positive":20} ==> {FP:5d}' ,
      f'{"True Negative":20} ==> {TN:5d}' ,
      f'{"False Negative":20} ==> {FN:5d}' ,
      "\n"
      f'{"Accuracy":20} ==> {round((((TP+TN)/(TP+TN+FP+FN))*100), 2):5df}' ,
      "\n\n"
      f'{"False Positive Words:":20} ==> {FP_words[0:10]:5d}' ,
      f'{"False Negative Words:":20} ==> {FN_words[0:10]:5d}' )                                                              
                                                                     
                                                                      
                                                                      
       

NameError: name 'eval_predict' is not defined

In [143]:
test = [1,2,3,4,2]

In [147]:
x = [int(i) for i,x in enumerate(test) if x == 2]



In [218]:
type(test.count(1))

int

In [153]:
l = 0
while l < len(x):
    for i in x:
        print(type(i))
        FP_words = test[i]
        l+=1
FP_words

<class 'int'>
<class 'int'>


2

#### Result measurement - Comparison Accuracy